In [10]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# 📊 FUNCIÓN LEER_HOJA ACTUALIZADA
# Funciona tanto localmente (con .env) como en GitHub Actions (con secrets)

import os
import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from dotenv import load_dotenv
import json

def configurar_google_sheets():
    """
    Configura la conexión a Google Sheets
    Funciona tanto localmente (con .env) como en GitHub Actions (con secrets)
    """
    # Cargar variables de entorno
    load_dotenv()
    
    # Configuración
    SHEET_ID = os.getenv('GOOGLE_SHEET_ID')
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    
    # Método 1: Intentar usar archivo JSON local (para ejecución local)
    SERVICE_ACCOUNT_FILE = os.getenv('GOOGLE_SERVICE_ACCOUNT_FILE')
    if SERVICE_ACCOUNT_FILE and os.path.exists(SERVICE_ACCOUNT_FILE):
        print("🔑 Usando archivo JSON local para autenticación")
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    else:
        # Método 2: Usar JSON desde variable de entorno (para GitHub Actions)
        SERVICE_ACCOUNT_JSON = os.getenv('GOOGLE_SERVICE_ACCOUNT_JSON')
        if SERVICE_ACCOUNT_JSON:
            print("🔑 Usando JSON desde variable de entorno para autenticación")
            # Crear credenciales desde el JSON
            service_account_info = json.loads(SERVICE_ACCOUNT_JSON)
            creds = Credentials.from_service_account_info(service_account_info, scopes=SCOPES)
        else:
            raise ValueError("❌ No se encontraron credenciales de Google. Configura GOOGLE_SERVICE_ACCOUNT_FILE o GOOGLE_SERVICE_ACCOUNT_JSON")
    
    # Crear servicio
    service = build('sheets', 'v4', credentials=creds)
    
    return service, SHEET_ID

# Configurar conexión global
service, SHEET_ID = configurar_google_sheets()
print(f"✅ Conectado a Google Sheet: {SHEET_ID}")

def leer_hoja(nombre_hoja='REGISTRO_DIARIO'):
    """
    Lee TODA la hoja completa desde Google Sheets, sin importar cuántas filas tenga.
    Se adapta automáticamente al crecimiento de datos y maneja celdas vacías.
    
    Args:
        nombre_hoja (str): Nombre de la hoja a leer
        
    Returns:
        pandas.DataFrame: DataFrame con los datos de la hoja o None si hay error
    """
    try:
        # Leer toda la hoja (sin especificar rango)
        range_name = f'{nombre_hoja}'
        
        result = service.spreadsheets().values().get(
            spreadsheetId=SHEET_ID, 
            range=range_name
        ).execute()
        
        values = result.get('values', [])
        
        if values:
            if len(values) > 1:
                # Obtener encabezados y número de columnas esperadas
                headers = values[0]
                num_columns = len(headers)
                
                # Normalizar todas las filas para que tengan el mismo número de columnas
                data_rows = []
                for row in values[1:]:
                    # Si la fila tiene menos columnas, completar con cadenas vacías
                    if len(row) < num_columns:
                        row_completa = row + [''] * (num_columns - len(row))
                        data_rows.append(row_completa)
                    else:
                        data_rows.append(row)
                
                df = pd.DataFrame(data_rows, columns=headers)
            else:
                df = pd.DataFrame(values)
            
            print(f"✅ Se leyeron {len(df)} filas de {nombre_hoja}")
            print(f"📊 Columnas: {list(df.columns)}")
            return df
        else:
            print("❌ No se encontraron datos")
            return None
            
    except Exception as e:
        print(f"❌ Error al leer la hoja {nombre_hoja}: {e}")
        return None

print("🎯 Función leer_hoja() lista para usar")

🔑 Usando archivo JSON local para autenticación
✅ Conectado a Google Sheet: 14AtL1_MHWaN1JR_dbbddif9w0ujlx4wFrEuelmv6gfs
🎯 Función leer_hoja() lista para usar


# 📧 CONFIGURACIÓN REQUERIDA PARA GMAIL

Para que funcione el envío de correos, necesitas configurar las siguientes variables en tu archivo `.env`:

```env
# Configuración Gmail
GMAIL_USER=tu-email@gmail.com
GMAIL_APP_PASSWORD=tu-contraseña-de-aplicacion
```

## 🔑 ¿Cómo obtener la contraseña de aplicación de Gmail?

1. **Ir a tu cuenta de Google** → Configuración de seguridad
2. **Activar la verificación en 2 pasos** (si no la tienes)
3. **Generar una contraseña de aplicación**:
   - Ve a "Contraseñas de aplicaciones"
   - Selecciona "Correo" y "Otro"
   - Copia la contraseña de 16 caracteres generada
4. **Agregar al archivo .env** sin espacios ni guiones

⚠️ **IMPORTANTE**: Usa la contraseña de aplicación, NO tu contraseña normal de Gmail.

# REGISTRO CALENDARIO

In [12]:
Registro_calendario=leer_hoja("REGISTRO_CALENDARIO")

✅ Se leyeron 809 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime', 'ID_Registro']


In [13]:
Registro_calendario.tail()

,Colaborador,HoraEntrada,HoraSalida,FechaEntrada,FechaSalida,Minutos,Minutos_extras,Minutos_normales,ID_Calendario,Alerta,Descripcion,Observacion,Extratime,ID_Registro
804,Andrea Yuqui,16:01:04,,10/9/2025,,,,,RCL-000000805,,,,,b570ba9c
805,Paulina Cunia,16:40:24,19:31:10,10/9/2025,10/9/2025,,,,RCL-000000806,,,,,e48e8d15
806,Edison Yuqui,18:06:43,,10/9/2025,,,,,RCL-000000807,,,,,3239842f
807,Lucero Ocaña,20:15:37,20:54:21,10/9/2025,10/9/2025,,,,RCL-000000808,,,,,ed76be90
808,Rosa Clavo,20:15:52,20:53:22,10/9/2025,10/9/2025,,,,RCL-000000809,,,,,b1a4e4d0


# Poner entradas

In [14]:
# buscar nuevos registros
# Usar iloc para evitar KeyError por índices negativos
Registro_calendario=leer_hoja("REGISTRO_CALENDARIO")
Registro_diario=leer_hoja("REGISTRO_DIARIO")
FilasCalendario=len(Registro_calendario)
filasDiarios=len(Registro_diario)
ultimoID = Registro_calendario.loc[FilasCalendario - 1, "ID_Registro"]
IndexNuevo = 0
for i in range(filasDiarios):
    if ultimoID == Registro_diario.loc[filasDiarios - 1 - i, "ID"]:
        IndexNuevo = filasDiarios - 1 - i
        break
IndexNuevo=IndexNuevo+1
Entradas=[]
i=0
for j in range(IndexNuevo, filasDiarios):
    if Registro_diario.loc[j, "Etapa"] == "Entrada":
        entrada={
            "Colaborador": Registro_diario.loc[j, "Colaborador"],
            "HoraEntrada": Registro_diario.loc[j, "Hora"],
            "FechaEntrada": Registro_diario.loc[j, "Fecha"],
            "ID_Registro": Registro_diario.loc[j, "ID"]
        }
        Entradas.append(entrada)


Entradas[0]

✅ Se leyeron 809 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime', 'ID_Registro']
✅ Se leyeron 1636 filas de REGISTRO_DIARIO
📊 Columnas: ['ID', 'Fecha', 'Hora', 'Etapa', 'Colaborador', 'Día', 'Mes', 'Año', 'PERIODO', 'Rol', 'Descripcion', 'Captura de petición de horas extra', 'Observacion']


IndexError: list index out of range

In [ ]:
# 📝 FUNCIÓN SIMPLE PARA ESCRIBIR A GOOGLE SHEETS
def escribir_a_sheets(nombre_hoja, datos):
    """
    Escribe datos directamente a Google Sheets al final de la hoja
    
    Args:
        nombre_hoja (str): Nombre de la hoja
        datos (list): Lista de listas con los datos
    """
    try:
        # Obtener última fila
        result = service.spreadsheets().values().get(
            spreadsheetId=SHEET_ID,
            range=f'{nombre_hoja}!A:A'
        ).execute()
        
        ultima_fila = len(result.get('values', [])) + 1
        
        # Escribir datos
        service.spreadsheets().values().update(
            spreadsheetId=SHEET_ID,
            range=f'{nombre_hoja}!A{ultima_fila}',
            valueInputOption='USER_ENTERED',
            body={'values': datos}
        ).execute()
        
        print(f"✅ {len(datos)} filas escritas en {nombre_hoja}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

# Convertir tu array Entradas y escribir
datos_para_escribir = []
for entrada in Entradas:
    fila = [
        entrada["Colaborador"],
        entrada["HoraEntrada"],
        "",
        entrada["FechaEntrada"], 
        "",
        "",  # HoraSalida vacía
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        entrada["ID_Registro"]
    ]
    datos_para_escribir.append(fila)

# Escribir a la hoja
escribir_a_sheets("REGISTRO_CALENDARIO", datos_para_escribir)

✅ 0 filas escritas en REGISTRO_CALENDARIO


# FIn poner entradas

# Entrada sin salida

In [ ]:
Registro_calendario=leer_hoja("REGISTRO_CALENDARIO")
Registro_diario=leer_hoja("REGISTRO_DIARIO")
FilasCalendario=len(Registro_calendario)
filasDiarios=len(Registro_diario)
Salidas=[]
for i in range(FilasCalendario):
    if Registro_calendario.loc[FilasCalendario -1 -i,"HoraSalida"]=="":
        Colaborador=Registro_calendario.loc[FilasCalendario -1 -i,"Colaborador"] 
        ID_Registro=Registro_calendario.loc[FilasCalendario -1 -i,"ID_Registro"] 
        for j in range(filasDiarios):
            if Registro_diario.loc[filasDiarios -1 -j,"ID"]==ID_Registro:

                index=filasDiarios -1 -j
                for z in range (index, filasDiarios):
                    horasextra=""
                    if Registro_diario.loc[z,"Captura de petición de horas extra"]=="":
                        horasextra="No"
                    else:
                        horasextra="Si"
                    if Registro_diario.loc[z,"Etapa"]=="Salida" and Registro_diario.loc[z,"Colaborador"]==Colaborador:
                        salida={
                            "Colaborador": Colaborador,
                            "HoraSalida": Registro_diario.loc[z,"Hora"],
                            "FechaSalida": Registro_diario.loc[z,"Fecha"],
                            "ID_Calendario": Registro_calendario.loc[FilasCalendario -1 -i,"ID_Calendario"],
                            "Descripcion": Registro_diario.loc[z,"Descripcion"],
                            "Extratime": horasextra,
                            "ID_Registro": ID_Registro
                        }
                        Salidas.append(salida)
                        break

✅ Se leyeron 809 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime', 'ID_Registro']
✅ Se leyeron 1635 filas de REGISTRO_DIARIO
📊 Columnas: ['ID', 'Fecha', 'Hora', 'Etapa', 'Colaborador', 'Día', 'Mes', 'Año', 'PERIODO', 'Rol', 'Descripcion', 'Captura de petición de horas extra', 'Observacion']


In [ ]:
Salidas

In [ ]:
# 📝 FUNCIÓN OPTIMIZADA PARA ACTUALIZAR HORAS Y FECHAS DE SALIDA (SIN LÍMITE DE CUOTA)
def actualizar_salidas_sheets():
    """
    Actualiza las horas, fechas, descripción y horas extras de salida en REGISTRO_CALENDARIO usando ID_Calendario como referencia
    OPTIMIZADA: Usa batch update para evitar límites de cuota de Google
    """
    if not Salidas:
        print("❌ No hay salidas para actualizar")
        return False
    
    print(f"🔄 Preparando actualización de {len(Salidas)} salidas en REGISTRO_CALENDARIO...")
    
    try:
        # Leer toda la hoja para obtener los datos actuales
        result = service.spreadsheets().values().get(
            spreadsheetId=SHEET_ID,
            range='REGISTRO_CALENDARIO!A:N'  # Leer hasta columna N
        ).execute()
        
        values = result.get('values', [])
        
        if not values:
            print("❌ No se encontraron datos en REGISTRO_CALENDARIO")
            return False
        
        # Obtener encabezados
        headers = values[0]
        print(f"📊 Columnas encontradas: {headers}")
        
        # Encontrar índices de las columnas importantes
        try:
            col_id_calendario = headers.index('ID_Calendario')
            col_hora_salida = headers.index('HoraSalida')
            col_fecha_salida = headers.index('FechaSalida')
            col_descripcion = headers.index('Descripcion')
            col_extratime = headers.index('Extratime')
        except ValueError as e:
            print(f"❌ Error: No se encontró una columna requerida: {e}")
            return False
        
        # 🚀 PREPARAR TODAS LAS ACTUALIZACIONES EN UN SOLO BATCH
        batch_updates = []
        actualizaciones_encontradas = 0
        
        for salida in Salidas:
            id_calendario_buscado = salida["ID_Calendario"]
            nueva_hora_salida = salida["HoraSalida"]
            nueva_fecha_salida = salida["FechaSalida"]
            nueva_descripcion = salida["Descripcion"]
            nuevo_extratime = salida["Extratime"]
            colaborador = salida["Colaborador"]
            
            print(f"🔍 Preparando: {colaborador} - {id_calendario_buscado}")
            
            # Buscar la fila correspondiente
            fila_encontrada = None
            for i, fila in enumerate(values[1:], start=2):  # Empezar desde fila 2 (saltando encabezados)
                if len(fila) > col_id_calendario and fila[col_id_calendario] == id_calendario_buscado:
                    fila_encontrada = i
                    break
            
            if fila_encontrada:
                # Agregar las 4 actualizaciones al batch
                # Actualizar la hora de salida
                batch_updates.append({
                    'range': f'REGISTRO_CALENDARIO!{chr(ord("A") + col_hora_salida)}{fila_encontrada}',
                    'values': [[nueva_hora_salida]]
                })
                
                # Actualizar la fecha de salida
                batch_updates.append({
                    'range': f'REGISTRO_CALENDARIO!{chr(ord("A") + col_fecha_salida)}{fila_encontrada}',
                    'values': [[nueva_fecha_salida]]
                })
                
                # Actualizar la descripción
                batch_updates.append({
                    'range': f'REGISTRO_CALENDARIO!{chr(ord("A") + col_descripcion)}{fila_encontrada}',
                    'values': [[nueva_descripcion]]
                })
                
                # Actualizar extratime
                batch_updates.append({
                    'range': f'REGISTRO_CALENDARIO!{chr(ord("A") + col_extratime)}{fila_encontrada}',
                    'values': [[nuevo_extratime]]
                })
                
                actualizaciones_encontradas += 1
                print(f"   ✅ Preparado: Fila {fila_encontrada}")
                print(f"      🕐 Hora: {nueva_hora_salida}")
                print(f"      📅 Fecha: {nueva_fecha_salida}")
                print(f"      📝 Descripción: {nueva_descripcion}")
                print(f"      ⏰ Horas extra: {nuevo_extratime}")
            else:
                print(f"   ❌ No encontrado: {id_calendario_buscado}")
        
        if not batch_updates:
            print("❌ No se encontraron registros para actualizar")
            return False
        
        # 🚀 EJECUTAR TODAS LAS ACTUALIZACIONES EN UNA SOLA LLAMADA
        print(f"\n🔥 Ejecutando {len(batch_updates)} actualizaciones en batch...")
        
        batch_update_request = {
            'valueInputOption': 'USER_ENTERED',
            'data': batch_updates
        }
        
        service.spreadsheets().values().batchUpdate(
            spreadsheetId=SHEET_ID,
            body=batch_update_request
        ).execute()
        
        print(f"\n🎉 ¡ÉXITO! Se actualizaron {actualizaciones_encontradas} salidas en una sola operación")
        print(f"📊 Total de celdas actualizadas: {len(batch_updates)}")
        
        # Actualizar el DataFrame local
        global Registro_calendario
        Registro_calendario = leer_hoja("REGISTRO_CALENDARIO")
        print("🔄 DataFrame local actualizado")
        
        return True
        
    except Exception as e:
        print(f"❌ Error al actualizar salidas: {e}")
        return False

# 🚀 EJECUTAR LA FUNCIÓN OPTIMIZADA
if len(Salidas) > 0:
    print(f"🎯 Se encontraron {len(Salidas)} salidas para actualizar")
    
    # Mostrar preview de las salidas
    print("\n🔍 Preview de salidas a actualizar:")
    for i, salida in enumerate(Salidas[:3]):  # Mostrar solo las primeras 3
        print(f"   {i+1}. {salida['Colaborador']} - ID: {salida['ID_Calendario']}")
        print(f"      🕐 Hora: {salida['HoraSalida']}")
        print(f"      📅 Fecha: {salida['FechaSalida']}")
        print(f"      📝 Descripción: {salida['Descripcion']}")
        print(f"      ⏰ Horas extra: {salida['Extratime']}")
    
    if len(Salidas) > 3:
        print(f"   ... y {len(Salidas) - 3} más")
    
    print(f"\n💡 Usando batch update: {len(Salidas) * 4} actualizaciones en 1 sola llamada API")
    
    # Ejecutar actualización optimizada
    resultado = actualizar_salidas_sheets()
    
    if resultado:
        print("\n✅ ¡TODAS LAS SALIDAS SE ACTUALIZARON CORRECTAMENTE!")
        print("🚀 Sin problemas de cuota gracias a batch update")
    else:
        print("\n❌ Hubo problemas al actualizar las salidas")
else:
    print("ℹ️  No hay salidas pendientes para actualizar")

🎯 Se encontraron 25 salidas para actualizar

🔍 Preview de salidas a actualizar:
   1. Rosa Clavo - ID: RCL-000000809
      🕐 Hora: 20:53:22
      📅 Fecha: 10/9/2025
   2. Lucero Ocaña - ID: RCL-000000808
      🕐 Hora: 20:54:21
      📅 Fecha: 10/9/2025
   3. Paulina Cunia - ID: RCL-000000806
      🕐 Hora: 19:31:10
      📅 Fecha: 10/9/2025
   4. Patricia Sullca - ID: RCL-000000804
      🕐 Hora: 18:00:31
      📅 Fecha: 10/9/2025
   5. Paulina Cunia - ID: RCL-000000803
      🕐 Hora: 16:21:05
      📅 Fecha: 10/9/2025
   6. Rosa Clavo - ID: RCL-000000802
      🕐 Hora: 16:33:38
      📅 Fecha: 10/9/2025
   7. Lucero Ocaña - ID: RCL-000000801
      🕐 Hora: 19:37:08
      📅 Fecha: 10/9/2025
   8. Mariana Benites - ID: RCL-000000800
      🕐 Hora: 18:43:44
      📅 Fecha: 10/9/2025
   9. Rosa Clavo - ID: RCL-000000799
      🕐 Hora: 9:47:37
      📅 Fecha: 10/9/2025
   10. Stephany Perez - ID: RCL-000000797
      🕐 Hora: 16:08:33
      📅 Fecha: 10/9/2025
   11. Patricia Sullca - ID: RCL-000000796
   

# Fin poner entradas sin salida


# Calculo de minutos

In [ ]:
Registro_Calendario=leer_hoja("REGISTRO_CALENDARIO")
Horario_Laboral=leer_hoja("HORARIOLABORAL")

✅ Se leyeron 809 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime', 'ID_Registro']
✅ Se leyeron 20 filas de HORARIOLABORAL
📊 Columnas: ['DNI', 'Colaborador', 'dias', 'hora_entrada', 'hora_salida', 'ID']


In [ ]:
type(Registro_Calendario.loc[0,"HoraEntrada"])

str

In [19]:
# 📝 FUNCIÓN PARA ACTUALIZAR MINUTOS EN GOOGLE SHEETS
Registro_Calendario=leer_hoja("REGISTRO_CALENDARIO")
Horario_Laboral=leer_hoja("HORARIOLABORAL")


#  FUNCIÓN PARA PARSEAR DIFERENTES FORMATOS DE HORA
def parsear_hora(hora_str):
    """
    Convierte diferentes formatos de hora a datetime
    Maneja: '0:00', '6:29:47', '08:30:45', etc.
    """
    if not hora_str or hora_str == "":
        return None
        
    # Limpiar espacios
    hora_str = str(hora_str).strip()
    
    # Casos especiales para formatos como "0:00" -> "00:00:00"
    if hora_str.count(':') == 1:
        partes = hora_str.split(':')
        if len(partes) == 2:
            hora_str = f"{partes[0].zfill(2)}:{partes[1].zfill(2)}:00"
    
    # Casos especiales para formatos como "6:29:47" -> "06:29:47"
    if hora_str.count(':') == 2:
        partes = hora_str.split(':')
        if len(partes[0]) == 1:
            hora_str = f"{partes[0].zfill(2)}:{partes[1]}:{partes[2]}"
    
    # Intentar parsear con formato estándar
    try:
        return datetime.strptime(hora_str, "%H:%M:%S")
    except ValueError:
        print(f"⚠️ No se pudo parsear la hora: '{hora_str}'")
        return None

# �🔄 CÓDIGO CORREGIDO PARA CALCULAR MINUTOS
import locale
import pandas as pd
from datetime import datetime
import pandasql as ps
# Mapeo días inglés → español (más confiable)
dias_map = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes', 
    'Wednesday': 'Miercoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sabado',
    'Sunday': 'Domingo'
}

diasname = ["Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado", "Domingo"]
filas_calendario = len(Registro_Calendario)
minutos_calculados=[]
for i in range(filas_calendario):
    if Registro_Calendario.loc[filas_calendario -1 -i, "Minutos"] == "" and Registro_Calendario.loc[filas_calendario -1 -i, "HoraSalida"] != "":
        Colaborador = Registro_Calendario.loc[filas_calendario -1 -i, "Colaborador"]
        HoraEntrada = Registro_Calendario.loc[filas_calendario -1 -i, "HoraEntrada"]
        HoraSalida = Registro_Calendario.loc[filas_calendario -1 -i, "HoraSalida"]
        FechaEntrada = Registro_Calendario.loc[filas_calendario -1 -i, "FechaEntrada"]
        ID_Calendario = Registro_Calendario.loc[filas_calendario -1 -i, "ID_Calendario"]
        
        # Buscar horario del colaborador
        query = f'SELECT * FROM Horario_Laboral WHERE Colaborador="{Colaborador}"'
        resultado = ps.sqldf(query, locals())
        
        # Calcular minutos totales - SOLO CAMBIO AQUÍ
        HoraEntrada_dt = parsear_hora(HoraEntrada)
        HoraSalida_dt = parsear_hora(HoraSalida)
        
        if not HoraEntrada_dt or not HoraSalida_dt:
            continue
            
        Minutos = HoraSalida_dt - HoraEntrada_dt
        Minutos = int(Minutos.total_seconds() / 60)
        
        if Minutos < 0:
            Minutos = Minutos + 1440
        
        Minutos_normales = 0
        Minutos_extras = 0
        
        # Obtener día de la semana en español
        fecha_dt = pd.to_datetime(FechaEntrada)
        dia_ingles = fecha_dt.day_name()
        dia_espanol = dias_map.get(dia_ingles, dia_ingles)
        
        # Buscar horario correspondiente
        for j in range(len(resultado)):
            RangoDias = resultado.loc[j, "dias"].split("-")
            inicio = diasname.index(RangoDias[0])
            fin = diasname.index(RangoDias[-1])
            NewRangoDias = diasname[inicio:fin + 1]
            
            if dia_espanol in NewRangoDias:
                HoraInicio = resultado.loc[j, "hora_entrada"]
                HoraFin = resultado.loc[j, "hora_salida"]
                # SOLO CAMBIO AQUÍ TAMBIÉN
                HoraInicio_dt = parsear_hora(HoraInicio)
                HoraFin_dt = parsear_hora(HoraFin)
                
                if not HoraInicio_dt or not HoraFin_dt:
                    continue
                
                Minutos_extras = 0
                if HoraInicio_dt > HoraEntrada_dt:
                    Minutos_extras += (HoraInicio_dt - HoraEntrada_dt).total_seconds() / 60
                if HoraFin_dt < HoraSalida_dt:
                    Minutos_extras += (HoraSalida_dt - HoraFin_dt).total_seconds() / 60
                    
                Minutos_normales = Minutos - Minutos_extras
                break
        
        Minutos_normales = int(Minutos_normales)
        Minutos_extras = int(Minutos_extras)
        
        calculo={
            "Minutos": Minutos,
            "Minutos_normales": Minutos_normales,
            "Minutos_extras": Minutos_extras,
            "ID_Calendario": ID_Calendario
        }

        minutos_calculados.append(calculo)

print("🎉 Cálculo de minutos completado")
len(minutos_calculados)

✅ Se leyeron 809 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime', 'ID_Registro']
✅ Se leyeron 20 filas de HORARIOLABORAL
📊 Columnas: ['DNI', 'Colaborador', 'dias', 'hora_entrada', 'hora_salida', 'ID']
🎉 Cálculo de minutos completado


25

In [21]:
# 📝 FUNCIÓN OPTIMIZADA PARA ACTUALIZAR MINUTOS EN GOOGLE SHEETS (SIN LÍMITE DE CUOTA)
def actualizar_minutos_por_id_calendario(minutos_calculados):
    """
    Actualiza los minutos en REGISTRO_CALENDARIO usando ID_Calendario como referencia
    OPTIMIZADA: Usa batch update para evitar límites de cuota de Google
    
    Args:
        minutos_calculados (list): Lista con diccionarios que contienen Minutos, Minutos_normales, Minutos_extras, ID_Calendario
    
    Returns:
        bool: True si todo se actualiza correctamente
    """
    if not minutos_calculados:
        print("❌ No hay datos para actualizar")
        return False
    
    print(f"🔄 Preparando actualización de {len(minutos_calculados)} registros en REGISTRO_CALENDARIO...")
    
    try:
        # Leer toda la hoja para obtener los datos actuales
        result = service.spreadsheets().values().get(
            spreadsheetId=SHEET_ID,
            range='REGISTRO_CALENDARIO!A:N'
        ).execute()
        
        values = result.get('values', [])
        
        if not values:
            print("❌ No se encontraron datos en REGISTRO_CALENDARIO")
            return False
        
        # Obtener encabezados
        headers = values[0]
        print(f"📊 Columnas encontradas: {headers}")
        
        # Encontrar índices de las columnas importantes
        try:
            col_id_calendario = headers.index('ID_Calendario')
            col_minutos = headers.index('Minutos')
            col_minutos_normales = headers.index('Minutos_normales')
            col_minutos_extras = headers.index('Minutos_extras')
        except ValueError as e:
            print(f"❌ Error: No se encontró una columna requerida: {e}")
            return False
        
        # 🚀 PREPARAR TODAS LAS ACTUALIZACIONES EN UN SOLO BATCH
        batch_updates = []
        actualizaciones_encontradas = 0
        
        for calculo in minutos_calculados:
            id_calendario_buscado = calculo["ID_Calendario"]
            minutos = calculo["Minutos"]
            minutos_normales = calculo["Minutos_normales"]
            minutos_extras = calculo["Minutos_extras"]
            
            print(f"🔍 Preparando: {id_calendario_buscado}")
            
            # Buscar la fila correspondiente
            fila_encontrada = None
            for i, fila in enumerate(values[1:], start=2):  # Empezar desde fila 2 (saltando encabezados)
                if len(fila) > col_id_calendario and fila[col_id_calendario] == id_calendario_buscado:
                    fila_encontrada = i
                    break
            
            if fila_encontrada:
                # Agregar las 3 actualizaciones al batch
                # Columna Minutos
                batch_updates.append({
                    'range': f'REGISTRO_CALENDARIO!{chr(ord("A") + col_minutos)}{fila_encontrada}',
                    'values': [[minutos]]
                })
                
                # Columna Minutos_normales
                batch_updates.append({
                    'range': f'REGISTRO_CALENDARIO!{chr(ord("A") + col_minutos_normales)}{fila_encontrada}',
                    'values': [[minutos_normales]]
                })
                
                # Columna Minutos_extras
                batch_updates.append({
                    'range': f'REGISTRO_CALENDARIO!{chr(ord("A") + col_minutos_extras)}{fila_encontrada}',
                    'values': [[minutos_extras]]
                })
                
                actualizaciones_encontradas += 1
                print(f"   ✅ Preparado: Fila {fila_encontrada} - {minutos} min ({minutos_normales}+{minutos_extras})")
            else:
                print(f"   ❌ No encontrado: {id_calendario_buscado}")
        
        if not batch_updates:
            print("❌ No se encontraron registros para actualizar")
            return False
        
        # 🚀 EJECUTAR TODAS LAS ACTUALIZACIONES EN UNA SOLA LLAMADA
        print(f"\n🔥 Ejecutando {len(batch_updates)} actualizaciones en batch...")
        
        batch_update_request = {
            'valueInputOption': 'USER_ENTERED',
            'data': batch_updates
        }
        
        service.spreadsheets().values().batchUpdate(
            spreadsheetId=SHEET_ID,
            body=batch_update_request
        ).execute()
        
        print(f"\n🎉 ¡ÉXITO! Se actualizaron {actualizaciones_encontradas} registros en una sola operación")
        print(f"📊 Total de celdas actualizadas: {len(batch_updates)}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error al actualizar minutos: {e}")
        return False

# 🚀 EJECUTAR LA ACTUALIZACIÓN OPTIMIZADA
if len(minutos_calculados) > 0:
    print(f"🎯 Se encontraron {len(minutos_calculados)} cálculos para enviar a Google Sheets")
    
    # Mostrar preview de los datos
    print("\n🔍 Preview de datos a actualizar:")
    for i, calculo in enumerate(minutos_calculados[:3]):  # Mostrar solo los primeros 3
        print(f"   {i+1}. ID_Calendario: {calculo['ID_Calendario']}")
        print(f"      🕐 Minutos: {calculo['Minutos']}")
        print(f"      📊 Normales: {calculo['Minutos_normales']}")
        print(f"      ⏰ Extras: {calculo['Minutos_extras']}")
    
    if len(minutos_calculados) > 3:
        print(f"   ... y {len(minutos_calculados) - 3} más")
    
    print(f"\n💡 Usando batch update: {len(minutos_calculados) * 3} actualizaciones en 1 sola llamada API")
    
    # Ejecutar actualización optimizada
    resultado = actualizar_minutos_por_id_calendario(minutos_calculados)
    
    if resultado:
        print("\n✅ ¡TODOS LOS DATOS SE ENVIARON CORRECTAMENTE!")
        print("🚀 Sin problemas de cuota gracias a batch update")
    else:
        print("\n❌ Hubo problemas al enviar los datos")
else:
    print("ℹ️  No hay cálculos pendientes para enviar")

🎯 Se encontraron 25 cálculos para enviar a Google Sheets

🔍 Preview de datos a actualizar:
   1. ID_Calendario: RCL-000000809
      🕐 Minutos: 37
      📊 Normales: 37
      ⏰ Extras: 0
   2. ID_Calendario: RCL-000000808
      🕐 Minutos: 38
      📊 Normales: 38
      ⏰ Extras: 0
   3. ID_Calendario: RCL-000000806
      🕐 Minutos: 170
      📊 Normales: 170
      ⏰ Extras: 0
   ... y 22 más

💡 Usando batch update: 75 actualizaciones en 1 sola llamada API
🔄 Preparando actualización de 25 registros en REGISTRO_CALENDARIO...
📊 Columnas encontradas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime', 'ID_Registro']
🔍 Preparando: RCL-000000809
   ✅ Preparado: Fila 810 - 37 min (37+0)
🔍 Preparando: RCL-000000808
   ✅ Preparado: Fila 809 - 38 min (38+0)
🔍 Preparando: RCL-000000806
   ✅ Preparado: Fila 807 - 170 min (170+0)
🔍 Preparando: RCL-000000804
   ✅